In [1]:
# using ast.literal_eval()
import sys
import ast
import os
import time
import pandas as pd
import pickle
import glob
import numpy as np
import statistics
from scipy.stats import mannwhitneyu
import math

In [2]:
with open("hPPIN_paths.dictionary", 'rb') as config_dictionary_file:
 
    # Step 3
    paths = pickle.load(config_dictionary_file)

In [3]:
hppin_paths = []


for key,val in paths.items() :
    for keys,vals in val.items():
        hppin_paths.append(vals)

In [4]:
statistics.mean(hppin_paths)

3.303814196484713

In [3]:
# create list
file2 = "common_nodes_hPPIN_STRING_PANCAN.txt"
combined_list = []
with open(file2) as IN:
    for line in IN:
        combined_list.append(line.rstrip().split("\t")[0])

In [4]:
def compute_sp(list1, list2, paths):
    total_sp = []
    for i in range(len(list1)):
        for j in range(len(list2)):
            if(list1[i] in paths.keys()):
                shortest_paths = paths[list1[i]]
                if(list2[j] in shortest_paths.keys()):
                    total_sp.append(shortest_paths[list2[j]])
    return({"distribution": total_sp,
            "mean": statistics.mean(total_sp),
            "median": statistics.median(total_sp)})

In [5]:
# Read Cancersea Data
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/hallmark"

filenames = os.listdir(folder_path)

hallmark = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hallmark[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hallmark[filename[:-4]].append(line.rstrip())

In [6]:
# read in HP sets
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets/"

filenames = os.listdir(folder_path)

hp_genesets = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hp_genesets[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hp_genesets[filename[:-4]].append(line.rstrip().split("\t")[0])

In [7]:
hallmark = hp_genesets

In [8]:
# read in HP matched controls set
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/"

filenames = glob.glob("/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/hPPIN*")
hp_genesets_matched = {}

for filename in filenames:
    name = filename.split("hPPIN_")[1][:-4]
    hp_genesets_matched[name] = list()

    with open(filename) as IN:
        for line in IN:
            hp_genesets_matched[name].append(line.rstrip().split("\t")[0])

In [9]:
a_mean = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_median = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_dist = pd.DataFrame(index = range(len(hallmark)),
                      columns = range(len(hp_genesets)))

In [10]:
i=0
for key1,value1 in hp_genesets.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_mean[i][j] = current_info["mean"]
        a_median[i][j] = current_info["median"]
        a_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [11]:
pd.DataFrame(a_median)

,0,1,2,3,4,5,6,7,8
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
7,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [12]:
pd.DataFrame(a_mean)

,0,1,2,3,4,5,6,7,8
0,2.968426,3.109072,3.070473,2.917557,2.900785,3.049051,3.016584,2.884369,3.152324
1,3.109072,3.194465,3.200153,3.072607,3.033303,3.176944,3.141510,3.009344,3.245121
2,3.070473,3.200153,3.155611,3.014669,2.993305,3.142599,3.112511,2.978027,3.246950
3,2.917557,3.072607,3.014669,2.805982,2.829761,2.998948,2.968329,2.838295,3.118893
4,2.900785,3.033303,2.993305,2.829761,2.795046,2.977856,2.942431,2.805015,3.080733
5,3.049051,3.176944,3.142599,2.998948,2.977856,3.120046,3.091141,2.957375,3.221413
6,3.016584,3.141510,3.112511,2.968329,2.942431,3.091141,3.054401,2.922487,3.184416
7,2.884369,3.009344,2.978027,2.838295,2.805015,2.957375,2.922487,2.766924,3.051744
8,3.152324,3.245121,3.246950,3.118893,3.080733,3.221413,3.184416,3.051744,3.284172


In [39]:
pd.DataFrame(a_mean, columns = hallmark.keys(), index = hp_genesets.keys()).to_csv("hPPIN_meanSP_HP_Phenotypes_hallmark_9by9.csv")

In [14]:
a_matched_mean = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_median = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_dist = pd.DataFrame(index = range(len(hallmark)),
                              columns = range(len(hp_genesets_matched)))

In [15]:
i=0
for key1,value1 in hp_genesets_matched.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_matched_mean[i][j] = current_info["mean"]
        a_matched_median[i][j] = current_info["median"]
        a_matched_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [16]:
pd.DataFrame(a_matched_median)

,0,1,2,3,4,5,6,7,8
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
7,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [17]:
pd.DataFrame(a_matched_mean)

,0,1,2,3,4,5,6,7,8
0,3.128623,3.239512,3.219909,3.093442,3.058000,3.198791,3.164010,3.024539,3.280218
1,3.150410,3.261115,3.243042,3.115435,3.079692,3.221848,3.186400,3.044163,3.301935
2,3.288796,3.402993,3.382062,3.251375,3.216055,3.361355,3.324918,3.182310,3.443398
3,3.155814,3.268841,3.249317,3.120547,3.086906,3.229099,3.192039,3.052614,3.309170
4,3.370990,3.486346,3.463483,3.339622,3.302793,3.443207,3.407768,3.257423,3.526971
5,3.181917,3.296328,3.275822,3.145635,3.111634,3.254633,3.218950,3.078326,3.336456
6,3.145913,3.259683,3.240912,3.112281,3.079233,3.219607,3.183811,3.042209,3.299537
7,3.115951,3.230676,3.209408,3.081259,3.044754,3.188498,3.152633,3.009593,3.272089
8,3.149370,3.265870,3.244260,3.113638,3.079560,3.223342,3.186270,3.043453,3.304157


In [40]:
pd.DataFrame(a_matched_mean, columns = hallmark.keys(), index = hp_genesets_matched.keys()).to_csv("hPPIN_meanSP_HPmatched_Phenotypes_hallmark_9by9.csv")

In [19]:
mannwhitneyu_p = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_matched_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))

In [20]:
for i in range(len(hp_genesets)):
    for j in range(len(hallmark)):
        actual_dist = a_dist[i][j]
        control_dist = a_matched_dist[i][j]
        res = mannwhitneyu(actual_dist, control_dist, alternative="less")
        mannwhitneyu_p[i][j] = res.pvalue
        test_dist[i][j] = statistics.mean(actual_dist)
        test_matched_dist[i][j] = statistics.mean(control_dist)

In [21]:
hallmark.keys()

dict_keys(['downregulated.wound', 'upregulated.stress', 'downregulated.regen', 'msigdb.wound', 'msigdb.regen', 'upregulated.regen', 'upregulated.wound', 'msigdb.stress', 'downregulated.stress'])

In [22]:
pvals = pd.DataFrame(mannwhitneyu_p, columns = hallmark.keys(), index = hp_genesets.keys())

In [23]:
pvals

,downregulated.wound,upregulated.stress,downregulated.regen,msigdb.wound,msigdb.regen,upregulated.regen,upregulated.wound,msigdb.stress,downregulated.stress
downregulated.wound,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
upregulated.stress,5.013794e-86,2.465821e-265,1.807337e-89,2.705967e-61,8.573960e-28,3.198616e-103,1.752279e-112,2.871169e-76,1.812471e-170
downregulated.regen,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
msigdb.wound,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
msigdb.regen,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
upregulated.regen,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
upregulated.wound,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
msigdb.stress,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
downregulated.stress,1.000000e+00,3.527066e-06,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.587826e-02


In [41]:
pvals.to_csv("hPPIN_matched_pvals_hallmark_9by9.csv")

In [35]:
LogFC = np.zeros(shape=(len(hp_genesets), len(hallmark)))
for i in range(len(hp_genesets)):
    for j in range(len(hallmark)):
        test = test_dist[i][j]
        matched =test_matched_dist[i][j]
        LogFC[i][j] = math.log((test/matched))

In [36]:
LFC = pd.DataFrame(LogFC, columns = hallmark.keys(), index = hp_genesets.keys())

In [37]:
LFC

,downregulated.wound,upregulated.stress,downregulated.regen,msigdb.wound,msigdb.regen,upregulated.regen,upregulated.wound,msigdb.stress,downregulated.stress
downregulated.wound,-0.052561,-0.041098,-0.047521,-0.058538,-0.052780,-0.047942,-0.047715,-0.047453,-0.039770
upregulated.stress,-0.013208,-0.020650,-0.013313,-0.013842,-0.015178,-0.014035,-0.014188,-0.011504,-0.017356
downregulated.regen,-0.068690,-0.061457,-0.069303,-0.075588,-0.071777,-0.067294,-0.066015,-0.066346,-0.058743
msigdb.wound,-0.078500,-0.061909,-0.074955,-0.106255,-0.086977,-0.073942,-0.072661,-0.072795,-0.059219
msigdb.regen,-0.150225,-0.139202,-0.145897,-0.165665,-0.166920,-0.145200,-0.146821,-0.149527,-0.135272
upregulated.regen,-0.042653,-0.036889,-0.041519,-0.047754,-0.043945,-0.042232,-0.040515,-0.040084,-0.035089
upregulated.wound,-0.041979,-0.036926,-0.040425,-0.047357,-0.045444,-0.040719,-0.041495,-0.040149,-0.035513
msigdb.stress,-0.077228,-0.070969,-0.074825,-0.082135,-0.082011,-0.075248,-0.075803,-0.084069,-0.069715
downregulated.stress,0.000938,-0.006374,0.000829,0.001686,0.000381,-0.000599,-0.000582,0.002721,-0.006067


In [42]:
LFC.to_csv("hPPIN_matched_LFC_hallmark_9by9.csv")

In [ ]:
pd.DataFrame(test_dist)

In [ ]:
pd.DataFrame(test_matched_dist)